In [55]:
from torch_geometric.loader import DataLoader, ImbalancedSampler
from torch_geometric.transforms import FaceToEdge, OneHotDegree
import torchvision.transforms as transforms

from mantra.simplicial import SimplicialDataset
from mantra.transforms import (
    TriangulationToFaceTransform,
    OrientableToClassTransform,
    DegreeTransform,
)
from validation.validate_homology import validate_betti_numbers

import torch

In [56]:
tr = transforms.Compose(
    [
        TriangulationToFaceTransform(),
        FaceToEdge(remove_faces=False),
        DegreeTransform(),
        OrientableToClassTransform(),
        OneHotDegree(max_degree=8, cat=False),
    ]
)

dataset = SimplicialDataset(root="./data", transform=tr)


print()
print(f"Dataset: {dataset}:")
print("====================")
print(f"Number of graphs: {len(dataset)}")
print(f"Number of features: {dataset.num_features}")
print(f"Number of classes: {dataset.num_classes}")

data = dataset[0]  # Get the first graph object.

print()
print(data)
print("=============================================================")

# Gather some statistics about the first graph.
print(f"Number of nodes: {len(data.x)}")
print(f"Number of edges: {data.num_edges}")
print(f"Average node degree: {data.num_edges / len(data.x):.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Is undirected: {data.is_undirected()}")

print("=============================================================")
print(f"Number of orientable Manifolds: {sum(dataset.orientable)}")
print(
    f"Number of non-orientable Manifolds: {len(dataset) - sum(dataset.orientable)}"
)
print(
    f"Percentage: {sum(dataset.orientable) / len(dataset):.2f}, {(len(dataset) - sum(dataset.orientable)) / len(dataset):.2f}"
)


Dataset: SimplicialDataset(712):
Number of graphs: 712
Number of features: 9
Number of classes: 2

Data(dimension=[1], n_vertices=[1], torsion_coefficients=[3], betti_numbers=[3], orientable=[1], genus=[1], name='S^2', face=[3, 4], edge_index=[2, 12], x=[4, 9], y=[1])
Number of nodes: 4
Number of edges: 12
Average node degree: 3.00
Has isolated nodes: False
Has self-loops: False
Is undirected: True
Number of orientable Manifolds: 193
Number of non-orientable Manifolds: 519
Percentage: 0.27, 0.73


In [57]:
data.betti_numbers

[1, 0, 1]

In [58]:
dataset = dataset.shuffle()

train_dataset = dataset[:-150]
test_dataset = dataset[-150:]

print(f"Number of training graphs: {len(train_dataset)}")
print(f"Number of test graphs: {len(test_dataset)}")

Number of training graphs: 562
Number of test graphs: 150


In [59]:
train_loader = DataLoader(
    train_dataset, batch_size=10
)  # ,sampler=ImbalancedSampler(train_dataset))
test_loader = DataLoader(test_dataset, batch_size=10)


for batch in train_loader:
    break

batch.y

tensor([0, 0, 0, 1, 1, 0, 0, 1, 0, 1])

In [60]:
from operator import concat
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TAGConv, TransformerConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 3)

    def forward(self, batch):
        # 1. Obtain node embeddings
        x = self.conv1(batch.x, batch.edge_index)
        x = x.relu()
        x = self.conv2(x, batch.edge_index)
        x = x.relu()
        x = self.conv3(x, batch.edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch.batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x


model = GCN(hidden_channels=64)
print(model(batch))

tensor([[-4.5819e-02, -3.9331e-02, -4.2941e-02],
        [-3.7291e-02, -6.8504e-02, -5.4509e-03],
        [-2.7116e-02, -2.7440e-02,  2.0817e-05],
        [-3.9345e-02, -6.8072e-02, -8.1359e-02],
        [-2.8689e-02, -4.1856e-02, -2.5476e-02],
        [-8.1056e-03, -4.5064e-02,  2.9946e-02],
        [-3.0017e-02, -4.4348e-02, -1.5939e-02],
        [-3.4824e-03, -5.2788e-02, -1.7973e-02],
        [-1.5153e-02, -6.5248e-02, -6.7883e-02],
        [-6.2596e-02, -4.7315e-02, -9.9255e-03]], grad_fn=<AddmmBackward0>)


In [61]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
        out = model(data)  # Perform a single forward pass.
        loss = criterion(
            out, torch.tensor(data.betti_numbers, dtype=torch.float)
        )  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    losses = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data)

        losses += criterion(
            out, torch.tensor(data.betti_numbers, dtype=torch.float)
        )
    return losses


for epoch in range(1, 40):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(
        f"Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}"
    )

Epoch: 001, Train Acc: 8.8607, Test Acc: 1.9755
Epoch: 002, Train Acc: 6.2946, Test Acc: 1.3799
Epoch: 003, Train Acc: 7.3765, Test Acc: 1.8066
Epoch: 004, Train Acc: 7.7206, Test Acc: 1.9605
Epoch: 005, Train Acc: 5.7995, Test Acc: 1.4090
Epoch: 006, Train Acc: 6.5125, Test Acc: 1.5056
Epoch: 007, Train Acc: 6.4604, Test Acc: 1.6126
Epoch: 008, Train Acc: 6.6463, Test Acc: 1.9684
Epoch: 009, Train Acc: 4.9964, Test Acc: 1.2566
Epoch: 010, Train Acc: 5.0923, Test Acc: 1.2564
Epoch: 011, Train Acc: 4.7138, Test Acc: 1.1211
Epoch: 012, Train Acc: 5.6023, Test Acc: 1.5135
Epoch: 013, Train Acc: 4.6682, Test Acc: 1.1419
Epoch: 014, Train Acc: 4.7652, Test Acc: 1.2088
Epoch: 015, Train Acc: 5.6908, Test Acc: 1.3544
Epoch: 016, Train Acc: 5.1775, Test Acc: 1.2503
Epoch: 017, Train Acc: 4.4483, Test Acc: 1.0854
Epoch: 018, Train Acc: 4.8420, Test Acc: 1.2799
Epoch: 019, Train Acc: 5.3053, Test Acc: 1.5795
Epoch: 020, Train Acc: 4.7974, Test Acc: 1.1136
Epoch: 021, Train Acc: 4.6213, Test Acc:

In [63]:
model.eval()
losses = 0
y_hat = []
y = []
for data in test_loader:  # Iterate in batches over the training/test dataset.
    out = model(data)
    y_hat.append(out.round().long())
    y.append(torch.tensor(data.betti_numbers))

y_hat = torch.vstack(y_hat)
y = torch.vstack(y)


incorrect = torch.count_nonzero(y - y_hat != 0).item()
correct = torch.count_nonzero(y - y_hat == 0).item()

print("predicted correct", correct)
print("predicted incorrect", incorrect)
print("percentage correct", correct / (correct + incorrect))

predicted incorrect 411
predicted incorrect 39
percentage correct 0.9133333333333333
